In [ ]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
import string
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy
from functools import reduce

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from MECPOCollection import MECPOCollection
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from CustomJSON import CustomEncoder, CustomWriter

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras

import scipy

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# ---------------------------------------------------------------------------------------------------

In [ ]:
fig_num=0

# save_dir_model_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230615\Models_00_05'
save_dir_model_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230615\Models'

In [ ]:
save_results = False
save_dir_model = None
if save_dir_model is None:
    save_dir_model = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir_model = os.path.join(save_dir_model_base, save_dir_model)
#-----
if not os.path.exists(save_dir_model) and save_results:
    os.makedirs(save_dir_model)

In [ ]:
combine_others=True

merged_df_full=pd.read_pickle(os.path.join(save_dir_model_base, 'merged_df_full.pkl'))
merged_df_no_outg=pd.read_pickle(os.path.join(save_dir_model_base, 'merged_df_no_outg.pkl'))
merged_df_no_outg_prstn=pd.read_pickle(os.path.join(save_dir_model_base, 'merged_df_no_outg_prstn.pkl'))
#-------------------------
# with open(os.path.join(save_dir_model_base, 'mecpo_coll_full.pkl'), 'rb') as handle:
#     mecpo_coll_full = pickle.load(handle)
# with open(os.path.join(save_dir_model_base, 'mecpo_coll_no_outg.pkl'), 'rb') as handle:
#     mecpo_coll_no_outg = pickle.load(handle)
# with open(os.path.join(save_dir_model_base, 'mecpo_coll_no_outg_prstn.pkl'), 'rb') as handle:
#     mecpo_coll_no_outg_prstn = pickle.load(handle)
#-------------------------
with open(os.path.join(save_dir_model_base, 'counts_series_full.pkl'), 'rb') as handle:
    counts_series_full = pickle.load(handle)
with open(os.path.join(save_dir_model_base, 'counts_series_no_outg.pkl'), 'rb') as handle:
    counts_series_no_outg = pickle.load(handle)
with open(os.path.join(save_dir_model_base, 'counts_series_no_outg_prstn.pkl'), 'rb') as handle:
    counts_series_no_outg_prstn = pickle.load(handle)
    
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
merged_df_no_outg_prstn.index.names     = ['trsf_pole_nb', 'no_outg_rec_nb']
counts_series_no_outg_prstn.index.names = ['trsf_pole_nb', 'no_outg_rec_nb']

In [ ]:
print(merged_df_full.shape)
print(merged_df_no_outg.shape)
print(merged_df_no_outg_prstn.shape)

In [ ]:
natsorted(merged_df_full.columns.get_level_values(1).unique().tolist())

### Initiate summary_dict

In [ ]:
summary_dict = dict()
#-------------------------
an_keys = natsorted(merged_df_full.columns.get_level_values(0).unique().tolist())
assert(an_keys==natsorted(merged_df_no_outg.columns.get_level_values(0).unique().tolist()))
assert(an_keys==natsorted(merged_df_no_outg_prstn.columns.get_level_values(0).unique().tolist()))
#-----
summary_dict['an_keys'] = an_keys
#-------------------------
n_top_reasons_to_inclue = 10
# n_top_reasons_to_inclue = None
#-----
summary_dict['n_top_reasons_to_inclue'] = n_top_reasons_to_inclue

# !!!!!!! No outage data have indices backwards from outages!!!!!


In [ ]:
merged_df_no_outg = merged_df_no_outg.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb'])
counts_series_no_outg = counts_series_no_outg.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb']).squeeze()
#-----
merged_df_no_outg_prstn = merged_df_no_outg_prstn.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb'])
counts_series_no_outg_prstn = counts_series_no_outg_prstn.reset_index().set_index(['no_outg_rec_nb', 'trsf_pole_nb']).squeeze()

In [ ]:
merged_df_full.columns.get_level_values(1).unique().tolist()

In [ ]:
if Path(save_dir_model_base).name=='Models' or Path(save_dir_model_base).name=='Models_00_05':
    is_norm=True
elif Path(save_dir_model_base).name=='Models_raw' or Path(save_dir_model_base).name=='Models_00_05_raw':
    is_norm=False
else:
    assert(0)

if n_top_reasons_to_inclue is not None:
    merged_df_full, [merged_df_no_outg, merged_df_no_outg_prstn] = MECPOCollection.get_top_reasons_subset_from_merged_cpo_df_and_project_from_others(
        merged_cpo_df=merged_df_full,
        other_dfs_w_counts_series=[ 
            [merged_df_no_outg, counts_series_no_outg], 
            [merged_df_no_outg_prstn, counts_series_no_outg_prstn]
        ], 
        how='per_mecpo_an', 
        n_reasons_to_include=n_top_reasons_to_inclue,
        combine_others=combine_others,
        output_combine_others_col='Other Reasons',
        SNs_tags=None, 
        is_norm=is_norm, 
        counts_series=counts_series_full
    )

In [ ]:
print(merged_df_full.shape)
print(merged_df_no_outg.shape)
print(merged_df_no_outg_prstn.shape)

In [ ]:
print(merged_df_full.shape)
print(merged_df_no_outg.shape)
print(merged_df_no_outg_prstn.shape)

In [ ]:
merged_df_full=MECPOCollection.get_total_event_counts_for_merged_cpo_df(merged_df_full)
merged_df_no_outg=MECPOCollection.get_total_event_counts_for_merged_cpo_df(merged_df_no_outg)
merged_df_no_outg_prstn=MECPOCollection.get_total_event_counts_for_merged_cpo_df(merged_df_no_outg_prstn)

In [ ]:
print(merged_df_full.shape)
print(merged_df_no_outg.shape)
print(merged_df_no_outg_prstn.shape)

In [ ]:
assert(len(set(merged_df_full.index).difference(set(counts_series_full.index)))==0)
assert(len(set(merged_df_no_outg.index).difference(set(counts_series_no_outg.index)))==0)
assert(len(set(merged_df_no_outg_prstn.index).difference(set(counts_series_no_outg_prstn.index)))==0)

merged_df_full = pd.merge(
    merged_df_full, 
    counts_series_full.to_frame(name=('nSNs', 'nSNs')), 
    left_index=True, right_index=True, how='inner')

merged_df_no_outg = pd.merge(
    merged_df_no_outg, 
    counts_series_no_outg.to_frame(name=('nSNs', 'nSNs')), 
    left_index=True, right_index=True, how='inner')

merged_df_no_outg_prstn = pd.merge(
    merged_df_no_outg_prstn, 
    counts_series_no_outg_prstn.to_frame(name=('nSNs', 'nSNs')), 
    left_index=True, right_index=True, how='inner')

In [ ]:
print(merged_df_full.shape)
print(merged_df_no_outg.shape)
print(merged_df_no_outg_prstn.shape)

In [ ]:
merged_df_full

In [ ]:
print(merged_df_full.shape)
print(merged_df_full.index.get_level_values(0).nunique())
print(merged_df_full.index.get_level_values(1).nunique())

In [ ]:
merged_df_full.index.get_level_values(1).value_counts()

In [ ]:
# merged_df_full['01-05 Days']
# merged_df_full['00-01 Days']

In [ ]:
merged_df_full

# =========================================================
# =========================================================

# EEMSP

In [ ]:
merge_eemsp = True
build_eemsp = False
#-----
summary_dict['merge_eemsp'] = merge_eemsp

In [ ]:
if merge_eemsp:
    trsf_pole_nbs = list(set(
        merged_df_full.index.get_level_values(1).unique().tolist()+
        merged_df_no_outg.index.get_level_values(1).unique().tolist()+
        merged_df_no_outg_prstn.index.get_level_values(1).unique().tolist()
    ))

In [ ]:
if merge_eemsp:
    if build_eemsp:
        conn_eemsp = Utilities.get_eemsp_oracle_connection()
        df_eemsp_OG = build_df_eemsp(conn_eemsp, trsf_pole_nbs, batch_size=1000, verbose=True, n_update=10)
        df_eemsp_OG.to_pickle(os.path.join(save_dir_model_base, 'df_eemsp_OG.pkl'))
    else:
        df_eemsp_OG = pd.read_pickle(os.path.join(save_dir_model_base, 'df_eemsp_OG.pkl'))

In [ ]:
if merge_eemsp:
    df_eemsp = df_eemsp_OG.copy()
    print(df_eemsp.shape)
    print(df_eemsp.columns)

In [ ]:
if merge_eemsp:
    # TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # Currently, simply taking the first entry wrt LOCATION_NB
    df_eemsp = df_eemsp.sort_values(by=['LOCATION_NB', 'INSTALL_DT'], ignore_index=True)
    df_eemsp = df_eemsp.groupby('LOCATION_NB', as_index=False).first()

In [ ]:
if merge_eemsp:
    cols_of_interest_eemsp = [
        'LOCATION_NB', 
        'MFGR_NM', 
        'INSTALL_DT', 
        'LAST_TRANS_DESC', 
        'EQTYPE_ID', 
        'COOLANT', 
        'INFO', 
        'KVA_SIZE',
        'PHASE_CNT', 
        'PRIM_VOLTAGE', 
        'PROTECTION', 
        'PRU_NUMBER', 
        'SEC_VOLTAGE', 
        'SPECIAL_CHAR', 
        'TAPS', 
        'XFTYPE'
    ]
    # df_eemsp=df_eemsp[df_eemsp['REMOVAL_DT'].isna()]
    df_eemsp=df_eemsp[cols_of_interest_eemsp]

In [ ]:
if merge_eemsp:
    assert(df_eemsp.shape[0]==df_eemsp['LOCATION_NB'].nunique())
    print(f"df_eemsp.shape[0]                 = {df_eemsp.shape[0]}")
    print(f"len(trsf_pole_nbs)                = {len(trsf_pole_nbs)}")
    print(f"Diff                              = {len(trsf_pole_nbs)-df_eemsp.shape[0]}")

In [ ]:
if merge_eemsp:
    #-------------------------
    print(f"merged_df_full.shape          = {merged_df_full.shape}")
    print(f"merged_df_no_outg.shape       = {merged_df_no_outg.shape}")
    print(f"merged_df_no_outg_prstn.shape = {merged_df_no_outg_prstn.shape}")
    #-------------------------
    df_eemsp = Utilities_df.prepend_level_to_MultiIndex(df_eemsp, level_val='EEMSP', axis=1)
    #-------------------------
    merged_df_full = pd.merge(
        merged_df_full, 
        df_eemsp.set_index(('EEMSP', 'LOCATION_NB')), 
        left_on=merged_df_full.index.get_level_values(1), 
        right_index=True, 
        how='inner'
    )
    #-------------------------
    merged_df_no_outg = pd.merge(
        merged_df_no_outg, 
        df_eemsp.set_index(('EEMSP', 'LOCATION_NB')), 
        left_on=merged_df_no_outg.index.get_level_values(1), 
        right_index=True, 
        how='inner'
    )
    #-------------------------
    merged_df_no_outg_prstn = pd.merge(
        merged_df_no_outg_prstn, 
        df_eemsp.set_index(('EEMSP', 'LOCATION_NB')), 
        left_on=merged_df_no_outg_prstn.index.get_level_values(1), 
        right_index=True, 
        how='inner'
    )
    #-------------------------
    print()
    print(f"merged_df_full.shape          = {merged_df_full.shape}")
    print(f"merged_df_no_outg.shape       = {merged_df_no_outg.shape}")
    print(f"merged_df_no_outg_prstn.shape = {merged_df_no_outg_prstn.shape}")

In [ ]:
# REMOVING SCHEDULED OUTAGES

print(f'merged_df_full.shape = {merged_df_full.shape}')

merged_df_full =  MECPODf.get_cpo_df_subset_excluding_mjr_mnr_causes( 
    cpo_df=merged_df_full, 
    mjr_mnr_causes_to_exclude=None, 
    mjr_causes_to_exclude=None,
    mnr_causes_to_exclude=['SCO', 'SO'], 
    outg_rec_nb_col='index'
)

print(f'merged_df_full.shape = {merged_df_full.shape}')

In [ ]:
merged_df_full.index.get_level_values(0).nunique()

In [ ]:
merged_df_full

In [ ]:
# no_outg_time_infos_df/no_outg_time_infos_prstn_df needed for splitting train/test according to time
build_no_outg_time_infos_dfs=False

if build_no_outg_time_infos_dfs:
#     ede_data_dir_no_outg_2020 = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20200101_20201231\EndEvents_NoOutg'
#     ede_data_dir_no_outg_2021 = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\EndEvents_NoOutg'
    ede_data_dir_no_outg_2022 = r'U:\CloudData\dovs_and_end_events_data\20230512\20220101_20221231\NoOutgs\EndEvents'
    #-------------------------
#     no_outg_time_infos_df_2020 = AMIEDE_DEV.get_no_outg_time_interval_infos_df_for_data_in_dir(ede_data_dir_no_outg_2020)
#     no_outg_time_infos_df_2021 = AMIEDE_DEV.get_no_outg_time_interval_infos_df_for_data_in_dir(ede_data_dir_no_outg_2021)
    no_outg_time_infos_df_2022 = AMIEDE_DEV.get_no_outg_time_interval_infos_df_for_data_in_dir(ede_data_dir_no_outg_2022)
    #-------------------------
#     no_outg_time_infos_df = pd.concat([no_outg_time_infos_df_2020, no_outg_time_infos_df_2021, no_outg_time_infos_df_2022])
    no_outg_time_infos_df = no_outg_time_infos_df_2022
    #-------------------------
    no_outg_time_infos_df.to_pickle(os.path.join(save_dir_model_base, 'no_outg_time_infos_df.pkl'))
else:
    no_outg_time_infos_df = pd.read_pickle(os.path.join(save_dir_model_base, 'no_outg_time_infos_df.pkl'))
#-----
if 'is_first_after_outg' in no_outg_time_infos_df.index.names:
    no_outg_time_infos_df = no_outg_time_infos_df.droplevel(level='is_first_after_outg', axis=0)
#--------------------------------------------------    
#--------------------------------------------------    
if build_no_outg_time_infos_dfs:
#     ede_data_dir_no_outg_prstn_2020 = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230301\20200101_20201231\NoOutgs_Pristine\EndEvents'
#     ede_data_dir_no_outg_prstn_2021 = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230301\20210101_20211231\NoOutgs_Pristine\EndEvents'
    ede_data_dir_no_outg_prstn_2022 = r'U:\CloudData\dovs_and_end_events_data\20230301\20220101_20221231\NoOutgs_Pristine\EndEvents'
    #-------------------------
#     no_outg_time_infos_df_prstn_2020 = AMIEDE_DEV.get_no_outg_time_interval_infos_df_for_data_in_dir(ede_data_dir_no_outg_prstn_2020)
#     no_outg_time_infos_df_prstn_2021 = AMIEDE_DEV.get_no_outg_time_interval_infos_df_for_data_in_dir(ede_data_dir_no_outg_prstn_2021)
    no_outg_time_infos_df_prstn_2022 = AMIEDE_DEV.get_no_outg_time_interval_infos_df_for_data_in_dir(ede_data_dir_no_outg_prstn_2022)
    #-------------------------
#     no_outg_time_infos_prstn_df = pd.concat([no_outg_time_infos_df_prstn_2020, no_outg_time_infos_df_prstn_2021, no_outg_time_infos_df_prstn_2022])
    no_outg_time_infos_prstn_df = no_outg_time_infos_df_prstn_2022
    #-------------------------
    no_outg_time_infos_prstn_df.to_pickle(os.path.join(save_dir_model_base, 'no_outg_time_infos_prstn_df.pkl'))
else:
    no_outg_time_infos_prstn_df = pd.read_pickle(os.path.join(save_dir_model_base, 'no_outg_time_infos_prstn_df.pkl'))
#-----
if 'is_first_after_outg' in no_outg_time_infos_prstn_df.index.names:
    no_outg_time_infos_prstn_df = no_outg_time_infos_prstn_df.droplevel(level='is_first_after_outg', axis=0)

# Add month info

In [ ]:
include_month = True
summary_dict['include_month'] = include_month

if include_month:
    merged_df_full_wd = DOVSOutages.append_outg_dt_off_ts_full_to_df(
        df=merged_df_full.copy(), 
        outg_rec_nb_idfr=('index', 'outg_rec_nb'), 
        dummy_col_levels_prefix='dummy_lvl_'
    )
    #-------------------------
    merged_df_no_outg_wd = merge_cpx_df_w_time_infos(
        cpx_df=merged_df_no_outg.copy(), 
        time_infos_df=no_outg_time_infos_df, 
        time_infos_drop_dupls_subset=['index', 't_min']
    )
    #-----
    merged_df_no_outg_prstn_wd = merge_cpx_df_w_time_infos(
        cpx_df=merged_df_no_outg_prstn.copy(), 
        time_infos_df=no_outg_time_infos_prstn_df, 
        time_infos_drop_dupls_subset=['index', 't_min']
    )
    #-------------------------
    if ('is_outg', 'is_outg') in merged_df_full_wd.columns:
        merged_df_full_wd = Utilities_df.move_cols_to_back(merged_df_full_wd, [('is_outg', 'is_outg')])
    if ('is_outg', 'is_outg') in merged_df_no_outg_wd.columns:
        merged_df_no_outg_wd = Utilities_df.move_cols_to_back(merged_df_no_outg_wd, [('is_outg', 'is_outg')])
    if ('is_outg', 'is_outg') in merged_df_no_outg_prstn_wd.columns:
        merged_df_no_outg_prstn_wd = Utilities_df.move_cols_to_back(merged_df_no_outg_prstn_wd, [('is_outg', 'is_outg')])
    #-------------------------
    # Change outage time to just month of outage
    merged_df_full_wd[('dummy_lvl_0', 'outg_month')] = merged_df_full_wd[('dummy_lvl_0', 'DT_OFF_TS_FULL')].dt.month

    merged_df_no_outg_wd[('dummy_lvl_0', 'outg_month')] = merged_df_no_outg_wd[('dummy_lvl_0', 't_min')].dt.month
    merged_df_no_outg_prstn_wd[('dummy_lvl_0', 'outg_month')] = merged_df_no_outg_prstn_wd[('dummy_lvl_0', 't_min')].dt.month

    #-------------------------
    merged_df_full_wd = merged_df_full_wd.drop(columns=[('dummy_lvl_0', 'DT_OFF_TS_FULL')])
    merged_df_no_outg_wd = merged_df_no_outg_wd.drop(columns=[('dummy_lvl_0', 't_min'), ('dummy_lvl_0', 't_max'), ('dummy_lvl_0', 'prem_nbs')])
    merged_df_no_outg_prstn_wd = merged_df_no_outg_prstn_wd.drop(columns=[('dummy_lvl_0', 't_min'), ('dummy_lvl_0', 't_max'), ('dummy_lvl_0', 'prem_nbs')])
    #-------------------------
    merged_df_full=merged_df_full_wd.copy()
    merged_df_no_outg=merged_df_no_outg_wd.copy()
    merged_df_no_outg_prstn=merged_df_no_outg_prstn_wd.copy()

# !~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~

In [ ]:
# Remove NETWORK and PRIMARY trsf_pole_nbs
# merged_df_no_outg=merged_df_no_outg.loc[~merged_df_no_outg.index.get_level_values(0).isin(['NETWORK', 'PRIMARY'])]
# merged_df_no_outg_prstn=merged_df_no_outg_prstn.loc[~merged_df_no_outg_prstn.index.get_level_values(0).isin(['NETWORK', 'PRIMARY'])]
merged_df_no_outg=merged_df_no_outg.loc[~merged_df_no_outg.index.get_level_values(1).isin(['NETWORK', 'PRIMARY'])]
merged_df_no_outg_prstn=merged_df_no_outg_prstn.loc[~merged_df_no_outg_prstn.index.get_level_values(1).isin(['NETWORK', 'PRIMARY'])]
#-----
merged_df_full=merged_df_full[~merged_df_full.index.get_level_values(1).isin(['NETWORK', 'PRIMARY'])]

In [ ]:
# Get DOVS info to be used for setting target values
merged_df_full_w_DOVS = DOVSOutages.append_outg_info_to_df(
    df=merged_df_full.copy(), 
    outg_rec_nb_idfr=('index', 'outg_rec_nb'), 
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
)
merged_df_full_w_DOVS=merged_df_full_w_DOVS[['outg_dummy_lvl_0']]
merged_df_full_w_DOVS[('is_outg', 'is_outg')]=0

In [ ]:
# Add 'from_outg' information so I can track how many are in target=1 and target=0 
merged_df_full[('from_outg', 'from_outg')]          = 1
merged_df_no_outg[('from_outg', 'from_outg')]       = 0
merged_df_no_outg_prstn[('from_outg', 'from_outg')] = 0

In [ ]:
an_keys

# Drop any time periods

In [ ]:
an_keys_to_drop = None
# an_keys_to_drop = ['00-01 Days']
# an_keys_to_drop = ['01-06 Days']

summary_dict['an_keys_to_drop'] = an_keys_to_drop

In [ ]:
if an_keys_to_drop is not None:
    merged_df_full          = merged_df_full.drop(columns=an_keys_to_drop, level=0)
    merged_df_no_outg       = merged_df_no_outg.drop(columns=an_keys_to_drop, level=0)
    merged_df_no_outg_prstn = merged_df_no_outg_prstn.drop(columns=an_keys_to_drop, level=0)

In [ ]:
date_0_train   = pd.to_datetime('2021-01-01')
date_1_train   = pd.to_datetime('2022-01-01')
#-----
date_0_test    = pd.to_datetime('2022-01-01')
date_1_test    = pd.to_datetime('2023-01-01')
#-----
date_0_HOLDOUT = pd.to_datetime('2022-06-01')
date_1_HOLDOUT = pd.to_datetime('2023-01-01')

# date_0_train   = pd.to_datetime('2021-01-01')
# date_1_train   = pd.to_datetime('2022-01-01')
# #-----
# date_0_test    = pd.to_datetime('2022-01-01')
# date_1_test    = pd.to_datetime('2022-06-01')
# #-----
# date_0_HOLDOUT = pd.to_datetime('2022-06-01')
# date_1_HOLDOUT = pd.to_datetime('2023-01-01')

# date_0_train   = pd.to_datetime('2020-01-01')
# date_1_train   = pd.to_datetime('2022-01-01')
# #-----
# date_0_test    = pd.to_datetime('2022-01-01')
# date_1_test    = pd.to_datetime('2022-01-01')
# #-----
# date_0_HOLDOUT = pd.to_datetime('2022-01-01')
# date_1_HOLDOUT = pd.to_datetime('2023-01-01')

In [ ]:
get_train_test_by_date = False
split_train_test_by_outg = True 
random_state = None
test_size=0.33

In [ ]:
# NOTE: Timestamp is not JSON serializable, hence the need for strftime below
summary_dict['date_0_train'] = date_0_train.strftime('%Y-%m-%d %H:%M:%S')
summary_dict['date_1_train'] = date_1_train.strftime('%Y-%m-%d %H:%M:%S')
#-----
summary_dict['date_0_test'] = date_0_test.strftime('%Y-%m-%d %H:%M:%S')
summary_dict['date_1_test'] = date_1_test.strftime('%Y-%m-%d %H:%M:%S')
#-----
summary_dict['date_0_HOLDOUT'] = date_0_HOLDOUT.strftime('%Y-%m-%d %H:%M:%S')
summary_dict['date_1_HOLDOUT'] = date_1_HOLDOUT.strftime('%Y-%m-%d %H:%M:%S')
#-----
summary_dict['get_train_test_by_date'] = get_train_test_by_date
summary_dict['split_train_test_by_outg'] = split_train_test_by_outg
#-----
summary_dict['random_state'] = random_state
summary_dict['test_size'] = test_size

In [ ]:
if get_train_test_by_date:
    merged_df_full_train = get_cpx_outg_df_subset_by_outg_datetime(
        cpx_outg_df=merged_df_full.copy(), 
        date_0 = date_0_train,
        date_1 = date_1_train, 
        outg_rec_nb_idfr='index', 
        return_notin_also=False
    )
    merged_df_full_test = get_cpx_outg_df_subset_by_outg_datetime(
        cpx_outg_df=merged_df_full.copy(), 
        date_0 = date_0_test,
        date_1 = date_1_test, 
        outg_rec_nb_idfr='index', 
        return_notin_also=False
    )
else:
    merged_df_full_train_test = get_cpx_outg_df_subset_by_outg_datetime(
        cpx_outg_df=merged_df_full.copy(), 
        date_0 = date_0_train,
        date_1 = date_1_test, 
        outg_rec_nb_idfr='index', 
        return_notin_also=False
    )
    #-----
    if split_train_test_by_outg:
        merged_df_full_train, merged_df_full_test = train_test_split_df_by_outage(
            df=merged_df_full_train_test, 
            outg_rec_nb_idfr=('index', 'outg_rec_nb'), 
            test_size=test_size, 
            random_state=random_state
        )
    else:
        merged_df_full_train, merged_df_full_test = train_test_split(
            merged_df_full_train_test, 
            test_size=test_size, 
            random_state=random_state
        )
#-------------------------    
merged_df_full_HOLDOUT = get_cpx_outg_df_subset_by_outg_datetime(
    cpx_outg_df=merged_df_full.copy(), 
    date_0 = date_0_HOLDOUT,
    date_1 = date_1_HOLDOUT, 
    outg_rec_nb_idfr='index', 
    return_notin_also=False
)

In [ ]:
if get_train_test_by_date:
    merged_df_no_outg_train = get_cpx_baseline_df_subset_by_datetime(
        cpx_bsln_df = merged_df_no_outg.copy(), 
        bsln_time_infos_df = no_outg_time_infos_df.copy(), 
        date_0 = date_0_train,
        date_1 = date_1_train, 
        bsln_time_infos_time_col='t_min', 
        return_notin_also=False, 
        merge_time_info_to_cpx_bsln_df=False
    )
    merged_df_no_outg_test = get_cpx_baseline_df_subset_by_datetime(
        cpx_bsln_df = merged_df_no_outg.copy(), 
        bsln_time_infos_df = no_outg_time_infos_df.copy(), 
        date_0 = date_0_test,
        date_1 = date_1_test, 
        bsln_time_infos_time_col='t_min', 
        return_notin_also=False, 
        merge_time_info_to_cpx_bsln_df=False
    )
else:
    merged_df_no_outg_train_test = get_cpx_baseline_df_subset_by_datetime(
        cpx_bsln_df = merged_df_no_outg.copy(), 
        bsln_time_infos_df = no_outg_time_infos_df.copy(), 
        date_0 = date_0_train,
        date_1 = date_1_test, 
        bsln_time_infos_time_col='t_min', 
        return_notin_also=False, 
        merge_time_info_to_cpx_bsln_df=False
    )
    #-----
    if split_train_test_by_outg:
        merged_df_no_outg_train, merged_df_no_outg_test = train_test_split_df_by_outage(
            df=merged_df_no_outg_train_test, 
            outg_rec_nb_idfr=('index', 'no_outg_rec_nb'), 
            test_size=test_size, 
            random_state=random_state
        )
    else:
        merged_df_no_outg_train, merged_df_no_outg_test = train_test_split(
            merged_df_no_outg_train_test, 
            test_size=test_size, 
            random_state=random_state
        )
#-------------------------   
merged_df_no_outg_HOLDOUT = get_cpx_baseline_df_subset_by_datetime(
    cpx_bsln_df = merged_df_no_outg.copy(), 
    bsln_time_infos_df = no_outg_time_infos_df.copy(), 
    date_0 = date_0_HOLDOUT,
    date_1 = date_1_HOLDOUT, 
    bsln_time_infos_time_col='t_min', 
    return_notin_also=False, 
    merge_time_info_to_cpx_bsln_df=False
)
#--------------------------------------------------   
if get_train_test_by_date:
    merged_df_no_outg_prstn_train = get_cpx_baseline_df_subset_by_datetime(
        cpx_bsln_df = merged_df_no_outg_prstn.copy(), 
        bsln_time_infos_df = no_outg_time_infos_prstn_df.copy(), 
        date_0 = date_0_train,
        date_1 = date_1_train, 
        bsln_time_infos_time_col='t_min', 
        return_notin_also=False, 
        merge_time_info_to_cpx_bsln_df=False
    )
    merged_df_no_outg_prstn_test = get_cpx_baseline_df_subset_by_datetime(
        cpx_bsln_df = merged_df_no_outg_prstn.copy(), 
        bsln_time_infos_df = no_outg_time_infos_prstn_df.copy(), 
        date_0 = date_0_test,
        date_1 = date_1_test, 
        bsln_time_infos_time_col='t_min', 
        return_notin_also=False, 
        merge_time_info_to_cpx_bsln_df=False
    )
else:
    merged_df_no_outg_prstn_train_test = get_cpx_baseline_df_subset_by_datetime(
        cpx_bsln_df = merged_df_no_outg_prstn.copy(), 
        bsln_time_infos_df = no_outg_time_infos_prstn_df.copy(), 
        date_0 = date_0_train,
        date_1 = date_1_test, 
        bsln_time_infos_time_col='t_min', 
        return_notin_also=False, 
        merge_time_info_to_cpx_bsln_df=False
    )
    if split_train_test_by_outg:
        merged_df_no_outg_prstn_train, merged_df_no_outg_prstn_test = train_test_split(
            merged_df_no_outg_prstn_train_test, 
            test_size=test_size, 
            random_state=random_state
        )
    else:
        merged_df_no_outg_prstn_train, merged_df_no_outg_prstn_test = train_test_split_df_by_outage(
            df=merged_df_no_outg_prstn_train_test, 
            outg_rec_nb_idfr=('index', 'no_outg_rec_nb'), 
            test_size=test_size, 
            random_state=random_state
        )
#-------------------------   
merged_df_no_outg_prstn_HOLDOUT = get_cpx_baseline_df_subset_by_datetime(
    cpx_bsln_df = merged_df_no_outg_prstn.copy(), 
    bsln_time_infos_df = no_outg_time_infos_prstn_df.copy(), 
    date_0 = date_0_HOLDOUT,
    date_1 = date_1_HOLDOUT, 
    bsln_time_infos_time_col='t_min', 
    return_notin_also=False, 
    merge_time_info_to_cpx_bsln_df=False
)

In [ ]:
addtnl_baseline_train = pd.concat([merged_df_no_outg_train, merged_df_no_outg_prstn_train])
# addtnl_baseline_train = addtnl_baseline_train.sample(frac=0.10)

addtnl_baseline_test = pd.concat([merged_df_no_outg_test, merged_df_no_outg_prstn_test])
# addtnl_baseline_test = addtnl_baseline_test.sample(frac=0.10)

addtnl_baseline_HOLDOUT = pd.concat([merged_df_no_outg_HOLDOUT, merged_df_no_outg_prstn_HOLDOUT])
# addtnl_baseline_HOLDOUT = addtnl_baseline_HOLDOUT.sample(frac=0.10)

In [ ]:
full_data_df_train   = pd.concat([merged_df_full_train, addtnl_baseline_train])
full_data_df_test    = pd.concat([merged_df_full_test, addtnl_baseline_test])
full_data_df_HOLDOUT = pd.concat([merged_df_full_HOLDOUT, addtnl_baseline_HOLDOUT])

#Shuffle the data
full_data_df_train   = full_data_df_train.sample(frac=1)
full_data_df_test    = full_data_df_test.sample(frac=1)
full_data_df_HOLDOUT = full_data_df_HOLDOUT.sample(frac=1)

full_data_df = pd.concat([full_data_df_train, full_data_df_test, full_data_df_HOLDOUT])

# =========================================================
# =========================================================

In [ ]:
#NOTE!!!!!!!!!!!!!
# If using _v2, SHOULD NOT SCALE events_period column!!!!!!!!!!!!

create_validation_set = False
val_size = 0.10 #w.r.t to train size (i.e., w.r.t 1.0-test_size)

run_scaler=True

run_PCA = False
pca_n_components=0.95

In [ ]:
summary_dict['create_validation_set'] = create_validation_set
summary_dict['val_size']              = val_size
summary_dict['run_scaler']            = run_scaler
summary_dict['run_PCA']               = run_PCA
summary_dict['pca_n_components']      = pca_n_components

In [ ]:
if merge_eemsp:
    full_data_df         = full_data_df.drop(columns=[('key_0', '')])
    full_data_df_train   = full_data_df_train.drop(columns=[('key_0', '')])
    full_data_df_test    = full_data_df_test.drop(columns=[('key_0', '')])
    full_data_df_HOLDOUT = full_data_df_HOLDOUT.drop(columns=[('key_0', '')])
    
    le = preprocessing.LabelEncoder()
    cols_to_encode = full_data_df['EEMSP'].columns
    for col in cols_to_encode:
        le.fit(full_data_df[('EEMSP', col)])
        #-----
        full_data_df_train[('EEMSP', col)]   = le.transform(full_data_df_train[('EEMSP', col)])
        full_data_df_test[('EEMSP', col)]    = le.transform(full_data_df_test[('EEMSP', col)])
        full_data_df_HOLDOUT[('EEMSP', col)] = le.transform(full_data_df_HOLDOUT[('EEMSP', col)])

# Set target values

In [ ]:
# fig_save_dir_base = r'C:\Users\s346557\Documents\AnalysisNote\X_Results\X_2_OutageMeterEvents\Figures\ConfusionMatrices'
# # fig_save_subdir = 'Target_eq_1_ALL_w_EEMSP'
# fig_save_subdir = 'Target_eq_1_ALL'
# # fig_save_subdir = 'trsf_nb_eq_LocID_w_EEMSP'
# fig_save_dir = os.path.join(fig_save_dir_base, fig_save_subdir)

In [ ]:
merged_df_full_w_DOVS

In [ ]:
outg_n_xfmrs = get_n_trsf_poles_per_outg(
    df=merged_df_full, 
    outg_rec_nb_idfr='index_0', 
    trsf_pole_nb_idfr='index_1'
)
outgs_w_single_xfmr = outg_n_xfmrs[outg_n_xfmrs==1].index.tolist()
#-------------------------

In [ ]:
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'LOCATION_ID'), 
#             value=merged_df_full_w_DOVS.index.get_level_values(1), 
#             comparison_operator='=='
#         ), 
#         dict(
#             column=('outg_dummy_lvl_0', 'EQUIP_TYP_NM'), 
#             value=xfmr_equip_typ_nms_of_interest, 
#             comparison_operator='isin'
#         )
#     ]
# )
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'LOCATION_ID'), 
#             value=merged_df_full_w_DOVS.index.get_level_values(1), 
#             comparison_operator='=='
#         )
#     ]
# )
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'MNR_CAUSE_NM'), 
#             value='EQUIPMENT FAILURE', 
#             comparison_operator='=='
#         )
#     ]
# )
#-----
slicer = DFSlicer()
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'EQUIP_TYP_NM'), 
#             value='CONDUCTOR OVERHEAD', 
#             comparison_operator='=='
#         )
#     ]
# )
#-----
# slicer = DFSlicer(
#     single_slicers = [
#         dict(
#             column=('outg_dummy_lvl_0', 'LOCATION_ID'), 
#             value=merged_df_full_w_DOVS.index.get_level_values(1), 
#             comparison_operator='=='
#         ), 
#         dict(
#             column=('outg_dummy_lvl_0', 'OUTG_REC_NB'), 
#             value=outgs_w_single_xfmr, 
#             comparison_operator='isin'
#         )
#     ]
# )
# merged_df_full_w_DOVS[('outg_dummy_lvl_0', 'OUTG_REC_NB')] = merged_df_full_w_DOVS.index.get_level_values(0)

In [ ]:
summary_dict['slicer'] = slicer.as_dict()

In [ ]:
merged_df_full_w_DOVS_i = slicer.set_simple_column_value(df=merged_df_full_w_DOVS.copy(), column=('is_outg', 'is_outg'), value=1)
full_outg_idxs_i = merged_df_full_w_DOVS_i[merged_df_full_w_DOVS_i[('is_outg', 'is_outg')]==1].index

#NOTE: To achieve exclusion equal to that in set_target_val_1_by_idx, one could use:
#        full_outg_idxs_exclude_i = merged_df_full_w_DOVS_i[merged_df_full_w_DOVS_i[('is_outg', 'is_outg')]==0].index
#      then use .drop(index=list(set(full_data_df_i.index).intersection(set(full_outg_idxs_exclude_i))))
#      However, as noted in the function, the methods in set_target_val_1_by_idx are probably safer/more robust

In [ ]:
remove_others_from_outages=False
summary_dict['remove_others_from_outages'] = remove_others_from_outages
#-------------------------
full_data_df_i         = full_data_df.copy()
full_data_df_train_i   = full_data_df_train.copy()
full_data_df_test_i    = full_data_df_test.copy()
full_data_df_HOLDOUT_i = full_data_df_HOLDOUT.copy()
#-------------------------
full_data_df_i = set_target_val_1_by_idx(
    df=full_data_df_i,
    val_1_idxs=full_outg_idxs_i,
    remove_others_from_outages=remove_others_from_outages, 
    target_col=('is_outg', 'is_outg'), 
    from_outg_col=('from_outg', 'from_outg')
)
#-----
full_data_df_train_i = set_target_val_1_by_idx(
    df=full_data_df_train_i,
    val_1_idxs=full_outg_idxs_i,
    remove_others_from_outages=remove_others_from_outages, 
    target_col=('is_outg', 'is_outg'), 
    from_outg_col=('from_outg', 'from_outg')
)
#-----
full_data_df_test_i = set_target_val_1_by_idx(
    df=full_data_df_test_i,
    val_1_idxs=full_outg_idxs_i,
    remove_others_from_outages=remove_others_from_outages, 
    target_col=('is_outg', 'is_outg'), 
    from_outg_col=('from_outg', 'from_outg')
)
#-----
full_data_df_HOLDOUT_i = set_target_val_1_by_idx(
    df=full_data_df_HOLDOUT_i,
    val_1_idxs=full_outg_idxs_i,
    remove_others_from_outages=remove_others_from_outages, 
    target_col=('is_outg', 'is_outg'), 
    from_outg_col=('from_outg', 'from_outg')
)

In [ ]:
#--------------------------------------------------
n_outg_target_1_train = full_data_df_train_i[('is_outg', 'is_outg')].sum()
n_outg_target_0_train = full_data_df_train_i[
    (full_data_df_train_i[('from_outg', 'from_outg')]==1) & 
    (full_data_df_train_i[('is_outg', 'is_outg')]==0)
].shape[0]
n_bsln_train = full_data_df_train_i[full_data_df_train_i[('from_outg', 'from_outg')]==0].shape[0]
assert(full_data_df_train_i.shape[0]==n_outg_target_1_train+n_outg_target_0_train+n_bsln_train)
pct_target_1_train = 100*n_outg_target_1_train/(n_outg_target_1_train+n_outg_target_0_train+n_bsln_train)
#-----
print('\n----- TRAIN -----')
print(f"# direct outages (target = 1):      {n_outg_target_1_train}")
print(f"# indirect outages (target = 0):    {n_outg_target_0_train}")
print(f"\t% direct = {100*n_outg_target_1_train/(n_outg_target_1_train+n_outg_target_0_train)}")
print(f"# additional baseline (target = 0): {n_bsln_train}")
print(f"%(target==1):                       {pct_target_1_train}%")

#--------------------------------------------------
n_outg_target_1_test = full_data_df_test_i[('is_outg', 'is_outg')].sum()
n_outg_target_0_test = full_data_df_test_i[
    (full_data_df_test_i[('from_outg', 'from_outg')]==1) & 
    (full_data_df_test_i[('is_outg', 'is_outg')]==0)
].shape[0]
n_bsln_test = full_data_df_test_i[full_data_df_test_i[('from_outg', 'from_outg')]==0].shape[0]
assert(full_data_df_test_i.shape[0]==n_outg_target_1_test+n_outg_target_0_test+n_bsln_test)
pct_target_1_test = 100*n_outg_target_1_test/(n_outg_target_1_test+n_outg_target_0_test+n_bsln_test)
#-----
print('\n----- TEST -----')
print(f"# direct outages (target = 1):      {n_outg_target_1_test}")
print(f"# indirect outages (target = 0):    {n_outg_target_0_test}")
print(f"\t% direct = {100*n_outg_target_1_test/(n_outg_target_1_test+n_outg_target_0_test)}")
print(f"# additional baseline (target = 0): {n_bsln_test}")
print(f"%(target==1):                       {pct_target_1_test}%")

#--------------------------------------------------
n_outg_target_1_HOLDOUT = full_data_df_HOLDOUT_i[('is_outg', 'is_outg')].sum()
n_outg_target_0_HOLDOUT = full_data_df_HOLDOUT_i[
    (full_data_df_HOLDOUT_i[('from_outg', 'from_outg')]==1) & 
    (full_data_df_HOLDOUT_i[('is_outg', 'is_outg')]==0)
].shape[0]
n_bsln_HOLDOUT = full_data_df_HOLDOUT_i[full_data_df_HOLDOUT_i[('from_outg', 'from_outg')]==0].shape[0]
assert(full_data_df_HOLDOUT_i.shape[0]==n_outg_target_1_HOLDOUT+n_outg_target_0_HOLDOUT+n_bsln_HOLDOUT)
pct_target_1_HOLDOUT = 100*n_outg_target_1_HOLDOUT/(n_outg_target_1_HOLDOUT+n_outg_target_0_HOLDOUT+n_bsln_HOLDOUT)
#-----
print('\n----- HOLDOUT -----')
print(f"# direct outages (target = 1):      {n_outg_target_1_HOLDOUT}")
print(f"# indirect outages (target = 0):    {n_outg_target_0_HOLDOUT}")
print(f"\t% direct = {100*n_outg_target_1_HOLDOUT/(n_outg_target_1_HOLDOUT+n_outg_target_0_HOLDOUT)}")
print(f"# additional baseline (target = 0): {n_bsln_HOLDOUT}")
print(f"%(target==1):                       {pct_target_1_HOLDOUT}%")

In [ ]:
if (
    full_data_df_train_i[('is_outg', 'is_outg')].sum()==0 or 
    full_data_df_test_i[('is_outg', 'is_outg')].sum()==0 or 
    full_data_df_HOLDOUT_i[('is_outg', 'is_outg')].sum()==0
):
    print('Not enough target value==1 in train, test, and/or holdout')
    print(f"#target==1 train:   {full_data_df_train_i[('is_outg', 'is_outg')].sum()}")
    print(f"#target==1 test:    {full_data_df_test_i[('is_outg', 'is_outg')].sum()}")
    print(f"#target==1 holdout: {full_data_df_HOLDOUT_i[('is_outg', 'is_outg')].sum()}")

In [ ]:
# min_pct_target_1 = 25
min_pct_target_1 = None
summary_dict['min_pct_target_1'] = min_pct_target_1
if min_pct_target_1 is not None:
    full_data_df_train_i = ensure_target_val_1_min_pct(
        df=full_data_df_train_i,
        min_pct=min_pct_target_1,
        target_col=('is_outg', 'is_outg'), 
        random_state=random_state
    )
    #-----
    full_data_df_test_i = ensure_target_val_1_min_pct(
        df=full_data_df_test_i,
        min_pct=min_pct_target_1,
        target_col=('is_outg', 'is_outg'), 
        random_state=random_state
    )
    #-----
    full_data_df_HOLDOUT_i = ensure_target_val_1_min_pct(
        df=full_data_df_HOLDOUT_i,
        min_pct=min_pct_target_1,
        target_col=('is_outg', 'is_outg'), 
        random_state=random_state
    )
    #-----

In [ ]:
#--------------------------------------------------
n_outg_target_1_train = full_data_df_train_i[('is_outg', 'is_outg')].sum()
n_outg_target_0_train = full_data_df_train_i[
    (full_data_df_train_i[('from_outg', 'from_outg')]==1) & 
    (full_data_df_train_i[('is_outg', 'is_outg')]==0)
].shape[0]
n_bsln_train = full_data_df_train_i[full_data_df_train_i[('from_outg', 'from_outg')]==0].shape[0]
assert(full_data_df_train_i.shape[0]==n_outg_target_1_train+n_outg_target_0_train+n_bsln_train)
pct_target_1_train = 100*n_outg_target_1_train/(n_outg_target_1_train+n_outg_target_0_train+n_bsln_train)
#-----
print('\n----- TRAIN -----')
print(f"# direct outages (target = 1):      {n_outg_target_1_train}")
print(f"# indirect outages (target = 0):    {n_outg_target_0_train}")
print(f"\t% direct = {100*n_outg_target_1_train/(n_outg_target_1_train+n_outg_target_0_train)}")
print(f"# additional baseline (target = 0): {n_bsln_train}")
print(f"%(target==1):                       {pct_target_1_train}%")

#--------------------------------------------------
n_outg_target_1_test = full_data_df_test_i[('is_outg', 'is_outg')].sum()
n_outg_target_0_test = full_data_df_test_i[
    (full_data_df_test_i[('from_outg', 'from_outg')]==1) & 
    (full_data_df_test_i[('is_outg', 'is_outg')]==0)
].shape[0]
n_bsln_test = full_data_df_test_i[full_data_df_test_i[('from_outg', 'from_outg')]==0].shape[0]
assert(full_data_df_test_i.shape[0]==n_outg_target_1_test+n_outg_target_0_test+n_bsln_test)
pct_target_1_test = 100*n_outg_target_1_test/(n_outg_target_1_test+n_outg_target_0_test+n_bsln_test)
#-----
print('\n----- TEST -----')
print(f"# direct outages (target = 1):      {n_outg_target_1_test}")
print(f"# indirect outages (target = 0):    {n_outg_target_0_test}")
print(f"\t% direct = {100*n_outg_target_1_test/(n_outg_target_1_test+n_outg_target_0_test)}")
print(f"# additional baseline (target = 0): {n_bsln_test}")
print(f"%(target==1):                       {pct_target_1_test}%")

#--------------------------------------------------
n_outg_target_1_HOLDOUT = full_data_df_HOLDOUT_i[('is_outg', 'is_outg')].sum()
n_outg_target_0_HOLDOUT = full_data_df_HOLDOUT_i[
    (full_data_df_HOLDOUT_i[('from_outg', 'from_outg')]==1) & 
    (full_data_df_HOLDOUT_i[('is_outg', 'is_outg')]==0)
].shape[0]
n_bsln_HOLDOUT = full_data_df_HOLDOUT_i[full_data_df_HOLDOUT_i[('from_outg', 'from_outg')]==0].shape[0]
assert(full_data_df_HOLDOUT_i.shape[0]==n_outg_target_1_HOLDOUT+n_outg_target_0_HOLDOUT+n_bsln_HOLDOUT)
pct_target_1_HOLDOUT = 100*n_outg_target_1_HOLDOUT/(n_outg_target_1_HOLDOUT+n_outg_target_0_HOLDOUT+n_bsln_HOLDOUT)
#-----
print('\n----- HOLDOUT -----')
print(f"# direct outages (target = 1):      {n_outg_target_1_HOLDOUT}")
print(f"# indirect outages (target = 0):    {n_outg_target_0_HOLDOUT}")
print(f"\t% direct = {100*n_outg_target_1_HOLDOUT/(n_outg_target_1_HOLDOUT+n_outg_target_0_HOLDOUT)}")
print(f"# additional baseline (target = 0): {n_bsln_HOLDOUT}")
print(f"%(target==1):                       {pct_target_1_HOLDOUT}%")

In [ ]:
if ('from_outg', 'from_outg') in full_data_df_i.columns.tolist():
    full_data_df_i = full_data_df_i.drop(columns=[('from_outg', 'from_outg')])
if ('from_outg', 'from_outg') in full_data_df_train_i.columns.tolist():
    full_data_df_train_i = full_data_df_train_i.drop(columns=[('from_outg', 'from_outg')])
if ('from_outg', 'from_outg') in full_data_df_test_i.columns.tolist():
    full_data_df_test_i = full_data_df_test_i.drop(columns=[('from_outg', 'from_outg')])
if ('from_outg', 'from_outg') in full_data_df_HOLDOUT_i.columns.tolist():
    full_data_df_HOLDOUT_i = full_data_df_HOLDOUT_i.drop(columns=[('from_outg', 'from_outg')])

In [ ]:
reduce_train_size = True
red_test_size = 0.95 #Amount kept will be 1.0-red_test_size
summary_dict['reduce_train_size'] = reduce_train_size
summary_dict['red_test_size'] = red_test_size
if reduce_train_size:
    if split_train_test_by_outg:
        full_data_df_train_i, _ = train_test_split_df_by_outage(
            df=full_data_df_train_i, 
            outg_rec_nb_idfr='index_0', 
            test_size=red_test_size, 
            random_state=random_state
        )
    else:
        full_data_df_train_i, _ = train_test_split(
            full_data_df_train_i, 
            test_size=red_test_size, 
            random_state=random_state
        )

# Prepare Data

In [ ]:
full_data_df_train_i.isna().sum()

In [ ]:
ugh = full_data_df_train_i.isna().sum()

In [ ]:
ugh[ugh>0]

In [ ]:
print(full_data_df_train_i.shape[0])
full_data_df_train_i = full_data_df_train_i.dropna()
print(full_data_df_train_i.shape[0])
print()
#-----
print(full_data_df_test_i.shape[0])
full_data_df_test_i = full_data_df_test_i.dropna()
print(full_data_df_test_i.shape[0])
print()
#-----
print(full_data_df_HOLDOUT_i.shape[0])
full_data_df_HOLDOUT_i = full_data_df_HOLDOUT_i.dropna()
print(full_data_df_HOLDOUT_i.shape[0])
print()
#-----

In [ ]:
full_data_df_test_i

In [ ]:
full_data_df_HOLDOUT_i

In [ ]:
# full_data_df_train_i   = full_data_df_train_i.drop(columns=('dummy_lvl_0', 'outg_month'))
# full_data_df_test_i    = full_data_df_test_i.drop(columns=('dummy_lvl_0', 'outg_month'))
# full_data_df_HOLDOUT_i = full_data_df_HOLDOUT_i.drop(columns=('dummy_lvl_0', 'outg_month'))

In [ ]:
set(full_data_df_train_i.columns.tolist()).symmetric_difference(set(full_data_df.columns.tolist()))

In [ ]:
#-------------------------
X_train_OG = full_data_df_train_i.iloc[:, :-1].copy()
y_train    = full_data_df_train_i.iloc[:, -1].copy()

X_test_OG = full_data_df_test_i.iloc[:, :-1].copy()
y_test    = full_data_df_test_i.iloc[:, -1].copy()

X_HOLDOUT = full_data_df_HOLDOUT_i.iloc[:, :-1]
y_HOLDOUT = full_data_df_HOLDOUT_i.iloc[:, -1]

In [ ]:
fig, axs = Plot_General.default_subplots(2,2, return_flattened_axes=True)
#-------------------------
sns.countplot(ax=axs[0], x = ('is_outg', 'is_outg'), data=full_data_df_i)
axs[0].set_title('Full Data');
axs[0].set_xlabel('Is Outage');
axs[0].set_ylim([1.05*x for x in axs[0].get_ylim()])
for p in axs[0].patches:
    pct = (p.get_height()/full_data_df_i.shape[0]).round(5)
    txt = f'{pct}%'
    txt_x = p.get_x() 
    txt_y = 1.025*p.get_height()
    axs[0].text(txt_x,txt_y,txt)
#-------------------------
sns.countplot(ax=axs[1], x = ('is_outg', 'is_outg'), data=y_train.to_frame())
axs[1].set_title('Training Data');
axs[1].set_xlabel('Is Outage');
axs[1].set_ylim([1.05*x for x in axs[1].get_ylim()])
for p in axs[1].patches:
    pct = (p.get_height()/y_train.shape[0]).round(5)
    txt = f'{pct}%'
    txt_x = p.get_x() 
    txt_y = 1.025*p.get_height()
    axs[1].text(txt_x,txt_y,txt)
#-------------------------
sns.countplot(ax=axs[2], x = ('is_outg', 'is_outg'), data=y_test.to_frame())
axs[2].set_title('Testing Data');
axs[2].set_xlabel('Is Outage');
axs[2].set_ylim([1.05*x for x in axs[2].get_ylim()])
for p in axs[2].patches:
    pct = (p.get_height()/y_test.shape[0]).round(5)
    txt = f'{pct}%'
    txt_x = p.get_x() 
    txt_y = 1.025*p.get_height()
    axs[2].text(txt_x,txt_y,txt)
#-------------------------
sns.countplot(ax=axs[3], x = ('is_outg', 'is_outg'), data=y_HOLDOUT.to_frame())
axs[3].set_title('Holdout Data');
axs[3].set_xlabel('Is Outage');
axs[3].set_ylim([1.05*x for x in axs[3].get_ylim()])
for p in axs[3].patches:
    pct = (p.get_height()/y_HOLDOUT.shape[0]).round(5)
    txt = f'{pct}%'
    txt_x = p.get_x() 
    txt_y = 1.025*p.get_height()
    axs[3].text(txt_x,txt_y,txt)

In [ ]:
create_validation_set

In [ ]:
run_PCA

In [ ]:
run_scaler

In [ ]:
#-------------------------
if create_validation_set:
    if split_train_test_by_outg:
        X_train_OG, X_val_OG, y_train, y_val = train_test_split_df_group(
            X=X_train_OG, 
            y=y_train, 
            groups=X_train_OG.index.get_level_values(0), 
            test_size=val_size, 
            random_state=random_state
        )
    else:
        X_train_OG, X_val_OG, y_train, y_val = train_test_split(X_train_OG, y_train, test_size=val_size, random_state=random_state)
#-------------------------
if run_scaler:
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_OG)
    #-----
    if create_validation_set:
        X_val   = scaler.transform(X_val_OG)
    X_test  = scaler.transform(X_test_OG)
    X_HOLDOUT   = scaler.transform(X_HOLDOUT)
else:
    X_train = X_train_OG
    if create_validation_set:
        X_val   = X_val_OG
    X_test  = X_test_OG

In [ ]:
X_train_OG

In [ ]:
X_train.shape

In [ ]:
X_train

In [ ]:
if run_PCA:
    # First, generate a PCA plot, showing the explained variance on the y-axis and number of components on x
    # This is simply to check that the number of components kept looks correct
    pca = PCA()
    pca.fit(X_train)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    plt.plot(cumsum)
    
    # Now, run the PCA with pca_n_components and perform transforms
    pca=PCA(n_components=pca_n_components)
    X_train = pca.fit_transform(X_train)
    print(f'PCA n-components       = {pca.n_components_}')
    print(f'PCA explained variance = {pca.explained_variance_ratio_.sum()}')
    #-----
    if create_validation_set:
        X_val      = pca.transform(X_val)
    X_test     = pca.transform(X_test)
    X_HOLDOUT  = pca.transform(X_HOLDOUT)

In [ ]:
X_train.shape

# Dumb Classifier

In [ ]:
dumb_clf = DumbClassifier()
cross_val_score(dumb_clf, X_train, y_train, cv=3, scoring='accuracy')

In [ ]:
# y_pred = dumb_clf.predict(X_test)
# print("ACCURACY  OF THE MODEL: ", accuracy_score(y_test, y_pred))
# print("PRECISION OF THE MODEL: ", precision_score(y_test, y_pred))
# print("RECALL    OF THE MODEL: ", recall_score(y_test, y_pred))

In [ ]:
# confusion_matrix(y_test, y_pred)

# PyTorch

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader

from torchmetrics.classification import BinaryPrecision

In [ ]:
class dataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.length = self.x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    def __len__(self):
        return self.length

In [ ]:
# class Net(nn.Module):
#     def __init__(self,input_shape):
#         super(Net,self).__init__()
#         self.fc1 = nn.Linear(input_shape,32)
#         self.fc2 = nn.Linear(32,64)
#         self.fc3 = nn.Linear(64,1)
#     def forward(self,x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = torch.sigmoid(self.fc3(x))
#         return x


# class Net(nn.Module):
#     def __init__(self,input_shape):
#         super(Net,self).__init__()
#         self.fc1 = nn.Linear(input_shape,1024)
#         self.fc2 = nn.Linear(1024,256)
#         self.fc3 = nn.Linear(256,128)
#         self.fc4 = nn.Linear(128,16)
#         self.fc5 = nn.Linear(16,4)
#         self.fc6 = nn.Linear(4,1)
#     def forward(self,x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = torch.relu(self.fc3(x))
#         x = torch.relu(self.fc4(x))
#         x = torch.relu(self.fc5(x))
#         x = torch.sigmoid(self.fc6(x))
#         return x


class Net(nn.Module):
    def __init__(self,input_shape):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_shape,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,16)
        self.fc5 = nn.Linear(16,4)
        self.fc6 = nn.Linear(4,1)
    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.sigmoid(self.fc6(x))
        return x

In [ ]:
trainset   = dataset(X_train,y_train)
testset    = dataset(X_test,y_test)
# holdoutset = dataset(X_HOLDOUT, y_HOLDOUT)

In [ ]:
#DataLoader
trainloader   = DataLoader(trainset,batch_size=64,shuffle=False)
testloader    = DataLoader(testset,batch_size=64,shuffle=False)
# holdoutloader = DataLoader(holdoutset,batch_size=64,shuffle=False)

In [ ]:
x = X_train
y = y_train.to_numpy()
print("shape of x: {}\nshape of y: {}".format(x.shape,y.shape))

sc = StandardScaler()
x = sc.fit_transform(x)

trainset = dataset(x,y)
# trainloader = DataLoader(trainset,batch_size=64,shuffle=False)
trainloader = DataLoader(trainset,batch_size=X_train.shape[0],shuffle=False)

#hyper parameters
# learning_rate = 0.1
# epochs = 700
# # Model , Optimizer, Loss
# model = Net(input_shape=x.shape[1])
# optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
# loss_fn = nn.BCELoss()

#hyper parameters
learning_rate = 0.0001
epochs = 10
# Model , Optimizer, Loss
model = Net(input_shape=X_train.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [ ]:
#forward loop
losses = []
accur = []
precisions = []
for i in range(epochs):
    print(i)
    for j,(x_t,y_t) in enumerate(trainloader):
    
        #calculate output
        output = model(x_t)

        #calculate loss
        loss = loss_fn(output,y_t.reshape(-1,1))

        #accuracy
        predicted = model(torch.tensor(X_train,dtype=torch.float32))
        acc = (predicted.reshape(-1).detach().numpy().round() == y_train).mean()
        precision = BinaryPrecision()(predicted.reshape(-1).detach(), torch.tensor(y_train.values))
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#     if i%50 == 0:
#         losses.append(loss)
#         accur.append(acc)
#         print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))
        losses.append(loss)
        accur.append(acc)
        precisions.append(precision)
        print("epoch {}\tloss : {}\t accuracy : {}\t precision : {}".format(i,loss,acc,precision))

In [ ]:
# #forward loop
# losses = []
# accur = []
# precisions = []

# loss_hist = [0]*epochs
# accuracy_hist=[0]*epochs

# for i in range(epochs):
#     print(i)
#     for j,(x_t,y_t) in enumerate(trainloader):
    
#         #calculate output
#         output = model(x_t)

#         #calculate loss
#         loss = loss_fn(output,y_t.reshape(-1,1))
        
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         loss_hist[i] += loss.item()*y_t.size(0)
#         is_correct = (torch.argmax(output, dim=1)==y_t).float()
#         accuracy_hist[i] += is_correct.sum()
#     loss_hist[i] /= len(trainloader.dataset)
#     accuracy_hist[i] /= len(trainloader.dataset)

In [ ]:
torch.from_numpy(X_test)

In [ ]:
y_pred = model(testset.x)

In [ ]:
testset.x

In [ ]:
y_pred

In [ ]:
torch.argmax(y_pred, dim=1).sum()

In [ ]:
correct = (torch.argmax(y_pred, dim=1)==y_test)

In [ ]:
correct.mean()

In [ ]:
# print('*****'*5)
# print('TESTING DATASET')
# print('*****'*5)
# print(f"#(target==1): {y_test.sum()}")
# print(f"#(target==0): {y_test.shape[0]-y_test.sum()}")
# print(f"%(target==1): {100*(y_test.sum()/(y_test.shape[0]))}")
# print('-----'*5)
# print("ACCURACY  OF THE MODEL: ", accuracy_score(y_test, y_pred.detach().numpy()))
# print("PRECISION OF THE MODEL: ", precision_score(y_test, y_pred.detach().numpy()))
# print("RECALL    OF THE MODEL: ", recall_score(y_test, y_pred.detach().numpy()))
# print()

In [ ]:
y_pred

In [ ]:
n_input_dim = X_train.shape[1]

#Layer size
n_hidden1 = 300  # Number of hidden nodes
n_hidden2 = 100
n_output =  1   # Number of output nodes = for binary classifier


class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.layer_1 = nn.Linear(n_input_dim, n_hidden1) 
        self.layer_2 = nn.Linear(n_hidden1, n_hidden2)
        self.layer_out = nn.Linear(n_hidden2, n_output) 
        
        
        self.relu = nn.ReLU()
        self.sigmoid =  nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(n_hidden1)
        self.batchnorm2 = nn.BatchNorm1d(n_hidden2)
        
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))
        
        return x
    

model = ChurnModel()
print(model)

In [ ]:
#Loss Computation
loss_func = nn.BCELoss()
#Optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 50

In [ ]:
model.train()
train_loss = []
for epoch in range(epochs):
    #Within each epoch run the subsets of data = batch sizes.
    for xb, yb in trainloader:
        y_pred = model(xb)            # Forward Propagation
        loss = loss_func(y_pred.squeeze(), yb)  # Loss Computation
        optimizer.zero_grad()         # Clearing all previous gradients, setting to zero 
        loss.backward()               # Back Propagation
        optimizer.step()              # Updating the parameters 
    #print("Loss in iteration :"+str(epoch)+" is: "+str(loss.item()))
    print(loss.item())
    train_loss.append(loss.item())
print('Last iteration loss value: '+str(loss.item()))

In [ ]:
plt.plot(train_loss)
plt.show()

In [ ]:
y_pred = model(torch.from_numpy(X_test.astype('float32')))

In [ ]:
y_pred.shape

In [ ]:
y_pred.reshape(-1).detach().numpy().shape

In [ ]:
(y_pred.reshape(-1).detach().numpy()>0.5).astype(int).shape

In [ ]:
y_test.shape

In [ ]:
((y_pred.reshape(-1).detach().numpy()>0.5).astype(int)==y_test).sum()

In [ ]:
((y_pred.reshape(-1).detach().numpy()>0.5).astype(int)==y_test).mean()

In [ ]:
y_pred = (y_pred.reshape(-1).detach().numpy()>0.5).astype(int)

In [ ]:
print('*****'*5)
print('TESTING DATASET')
print('*****'*5)
print(f"#(target==1): {y_test.sum()}")
print(f"#(target==0): {y_test.shape[0]-y_test.sum()}")
print(f"%(target==1): {100*(y_test.sum()/(y_test.shape[0]))}")
print('-----'*5)
print("ACCURACY  OF THE MODEL: ", accuracy_score(y_test, y_pred))
print("PRECISION OF THE MODEL: ", precision_score(y_test, y_pred))
print("RECALL    OF THE MODEL: ", recall_score(y_test, y_pred))
print()